In [2]:
import pandas as pd

In [4]:
cases = pd.read_csv('cases.csv')
labs = pd.read_csv('labs.csv')

In [9]:
cases.columns

Index(['caseid', 'subjectid', 'casestart', 'caseend', 'anestart', 'aneend',
       'opstart', 'opend', 'adm', 'dis', 'icu_days', 'death_inhosp', 'age',
       'sex', 'height', 'weight', 'bmi', 'asa', 'emop', 'department', 'optype',
       'dx', 'opname', 'approach', 'position', 'ane_type', 'preop_htn',
       'preop_dm', 'preop_ecg', 'preop_pft', 'preop_hb', 'preop_plt',
       'preop_pt', 'preop_aptt', 'preop_na', 'preop_k', 'preop_gluc',
       'preop_alb', 'preop_ast', 'preop_alt', 'preop_bun', 'preop_cr',
       'preop_ph', 'preop_hco3', 'preop_be', 'preop_pao2', 'preop_paco2',
       'preop_sao2', 'cormack', 'airway', 'tubesize', 'dltubesize', 'lmasize',
       'iv1', 'iv2', 'aline1', 'aline2', 'cline1', 'cline2', 'intraop_ebl',
       'intraop_uo', 'intraop_rbc', 'intraop_ffp', 'intraop_crystalloid',
       'intraop_colloid', 'intraop_ppf', 'intraop_mdz', 'intraop_ftn',
       'intraop_rocu', 'intraop_vecu', 'intraop_eph', 'intraop_phe',
       'intraop_epi', 'intraop_ca'],
     

In [13]:
merged = cases[['caseid', 'sex', 'age', 'height', 'weight', 'bmi', 'optype']].merge(labs, on='caseid')

In [14]:
merged

,caseid,sex,age,height,weight,bmi,optype,dt,name,result
0,1,M,77.0,160.2,67.5,26.3,Colorectal,594470,alb,2.90
1,1,M,77.0,160.2,67.5,26.3,Colorectal,399575,alb,3.20
2,1,M,77.0,160.2,67.5,26.3,Colorectal,12614,alb,3.40
3,1,M,77.0,160.2,67.5,26.3,Colorectal,137855,alb,3.60
4,1,M,77.0,160.2,67.5,26.3,Colorectal,399575,alt,12.00
...,...,...,...,...,...,...,...,...,...,...
928443,6388,F,47.0,150.0,51.5,22.9,Major resection,3503,sao2,100.00
928444,6388,F,47.0,150.0,51.5,22.9,Major resection,408770,wbc,3.28
928445,6388,F,47.0,150.0,51.5,22.9,Major resection,-32848,wbc,6.27
928446,6388,F,47.0,150.0,51.5,22.9,Major resection,-249820,wbc,7.66
